In [3]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import pandas as pd

import numpy as np
from sklearn.metrics import confusion_matrix
import itertools

import pathlib
import matplotlib.pyplot as plt

import pandas as pd

In [5]:
mv2 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", 
                   output_shape=[1001])
])

mv2_labels = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
mv2_labels = mv2_labels['background']

In [12]:
def get_mobilenet_pred(image_path, classifier, labels):
    img = Image.open(image_path)
    img = np.array(img)/255.0
    result = classifier.predict(img[np.newaxis, ...])
    predicted_label_indices = np.argpartition(result[0], -4)[-4:]
    return labels[predicted_label_indices]

In [16]:
# testing MobileNetV2 predictions

get_mobilenet_pred('per_imgs/dataset_3/cropped/train/pos/5_06-00047-c.png', mv2, mv2_labels)

44     alligator lizard
851          television
411               apron
364    three-toed sloth
Name: background, dtype: object

---

load input data

---

In [196]:
train_dir = pathlib.Path('per_imgs/dataset_3/cropped/train')
val_dir = pathlib.Path('per_imgs/dataset_3/cropped/val')
test_dir = pathlib.Path('per_imgs/dataset_3/cropped/test')
test_3_20_dir = pathlib.Path('per_imgs/dataset_3/resized/test_3_20')

In [197]:
train_images_dict = {
    'pos': list(train_dir.glob('pos/*')),
    'neg': list(train_dir.glob('neg/*')),
}

val_images_dict = {
    'pos': list(val_dir.glob('pos/*')),
    'neg': list(val_dir.glob('neg/*')),
}

test_images_dict = {
    'pos': list(test_dir.glob('pos/*')),
    'neg': list(test_dir.glob('neg/*')),
}

labels_dict = {
    'pos': [0, 1],
    'neg': [1, 0]
}

In [188]:
def assign_x_y(images_dict):
    
    X, y, file_names = [], [], []

    for label, images in images_dict.items():
        for image in images:
            try:
                img = cv2.imread(str(image))
                resized_img = cv2.resize(img, (224, 224))

                prep_img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

                X.append(prep_img)
                y.append(labels_dict[label])
                file_names.append(image)
            except:
                print(image)
            
            # convert to np array and scale
            
    X = np.array(X)
    y = np.array(y)
    
    return X, y, file_names

In [198]:
X_train, y_train, train_file_names = assign_x_y(train_images_dict)
X_val, y_val, val_file_names = assign_x_y(val_images_dict)
X_test, y_test, test_file_names = assign_x_y(test_images_dict)

per_imgs/dataset_3/cropped/test/pos/.DS_Store
per_imgs/dataset_3/cropped/test/neg/.DS_Store


---

building the model

---

1_3 had added dense layer with no preprocessing except for /255.

In [156]:
t_model_1_6 = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4', input_shape = (224, 224, 3), trainable = False),
    layers.Dense(1000, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(units = 2, activation = 'softmax')
])

In [157]:
t_model_1_6.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [158]:
callback = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta = 0.01, patience=4)

In [159]:
t_model_1_6.fit(X_train, y_train, epochs = 20)

Epoch 1/20
57/57 [==============================] - 51s 831ms/step - loss: 0.5546 - accuracy: 0.7531
Epoch 2/20
57/57 [==============================] - 57s 998ms/step - loss: 0.1707 - accuracy: 0.9332
Epoch 3/20
57/57 [==============================] - 54s 937ms/step - loss: 0.1243 - accuracy: 0.9529
Epoch 4/20
57/57 [==============================] - 54s 952ms/step - loss: 0.1192 - accuracy: 0.9573
Epoch 5/20
57/57 [==============================] - 52s 911ms/step - loss: 0.0873 - accuracy: 0.9616
Epoch 6/20
57/57 [==============================] - 48s 841ms/step - loss: 0.0953 - accuracy: 0.9682
Epoch 7/20
57/57 [==============================] - 47s 825ms/step - loss: 0.0684 - accuracy: 0.9781
Epoch 8/20
57/57 [==============================] - 47s 824ms/step - loss: 0.0664 - accuracy: 0.9725
Epoch 9/20
57/57 [==============================] - 46s 804ms/step - loss: 0.0589 - accuracy: 0.9752
Epoch 10/20
57/57 [==============================] - 45s 792ms/step - loss: 0.0533 - accura

KeyboardInterrupt: 

In [160]:
t_model_1_6.evaluate(X_val, y_val)

15/15 [==============================] - 13s 753ms/step - loss: 0.5054 - accuracy: 0.8877


[0.5053700804710388, 0.8876889944076538]

In [201]:
t_model_1_6.evaluate(X_test, y_test)

16/16 [==============================] - 12s 730ms/step - loss: 0.0950 - accuracy: 0.9469


[0.09500494599342346, 0.9468504190444946]

In [206]:
def df_preds(X, y, file_names):    
    
    preds = np.argmax(t_model_1_6.predict(X), axis = 1)
    
    preds_df = pd.DataFrame()

    preds_df['file'] = [str(x) for x in file_names]
    preds_df['file'] = preds_df['file'].apply(lambda x: x.split('/cropped/')[1])
    preds_df['label'] = np.argmax(y, axis = 1)
    preds_df['prediction'] = preds
    preds_df['correct_pred'] = np.where(preds_df['label'] == preds_df['prediction'], True, False)
#     bad_preds_df = preds_df[preds_df.correct_pred == False]
    return preds_df

In [208]:
test_df = df_preds(X_test, y_test, test_file_names)

In [209]:
test_df_pos = test_df[test_df.label == 1]
test_df_neg = test_df[test_df.label == 0]

In [213]:
print(test_df_pos.correct_pred.value_counts())
print(test_df_neg.correct_pred.value_counts())

True     148
False     24
Name: correct_pred, dtype: int64
True     333
False      3
Name: correct_pred, dtype: int64


In [27]:
from tensorflow.keras.models import load_model
model_1_4 = load_model('models/t_model_1_4.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [28]:
model_1_4.evaluate(X_val, y_val)

15/15 [==============================] - 14s 837ms/step - loss: 0.4481 - accuracy: 0.8575


[0.44809821248054504, 0.8574513792991638]

In [32]:
t_model_1_6.evaluate(X_val, y_val)

15/15 [==============================] - 15s 999ms/step - loss: 0.7486 - accuracy: 0.8726


[0.748550534248352, 0.8725702166557312]

In [33]:
model_1_4.evaluate(X_train, y_train)

61/61 [==============================] - 79s 1s/step - loss: 0.2219 - accuracy: 0.8783


[0.22194412350654602, 0.8783013820648193]

In [34]:
t_model_1_6.evaluate(X_train, y_train)

61/61 [==============================] - 85s 1s/step - loss: 0.1063 - accuracy: 0.9311


[0.10627804696559906, 0.9311237931251526]

In [219]:
t_model_1_6.save('models/model_1_6.h5')

In [161]:
preds = np.argmax(t_model_1_6.predict(X_val), axis = 1)

---

try a similar feature vector approach with a bird classifier

---

In [33]:
bird_classifier = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/experts/bit/r50x1/in21k/bird/1', input_shape = image_shape+(3,), trainable = False),
        tf.keras.layers.Dense(units = 2, activation = 'softmax')
])

In [34]:
bird_classifier.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [35]:
bird_classifier.fit(X_train, y_train, epochs = 10)

Epoch 1/10
52/52 [==============================] - 409s 7s/step - loss: 0.9307 - accuracy: 0.5325
Epoch 2/10
52/52 [==============================] - 359s 7s/step - loss: 0.3238 - accuracy: 0.8642
Epoch 3/10
52/52 [==============================] - 331s 6s/step - loss: 0.2425 - accuracy: 0.8945
Epoch 4/10
52/52 [==============================] - 350s 7s/step - loss: 0.2011 - accuracy: 0.9092
Epoch 5/10
52/52 [==============================] - 380s 7s/step - loss: 0.2046 - accuracy: 0.9032
Epoch 6/10
52/52 [==============================] - 366s 7s/step - loss: 0.2122 - accuracy: 0.8984
Epoch 7/10
52/52 [==============================] - 314s 6s/step - loss: 0.2009 - accuracy: 0.9024
Epoch 8/10
52/52 [==============================] - 296s 6s/step - loss: 0.1781 - accuracy: 0.9111
Epoch 9/10
52/52 [==============================] - 326s 6s/step - loss: 0.1764 - accuracy: 0.9046
Epoch 10/10
52/52 [==============================] - 321s 6s/step - loss: 0.1930 - accuracy: 0.8995


In [90]:
def get_transfer_pred(image_path, classifier = classifier):
    img = Image.open(image_path)
    img = np.array(img)/255.0
    result = classifier.predict(img[np.newaxis, ...])
    return result

def make_plot_predictions(test_batch, model):
    cm_plot_labels = ['neg', 'pos']
    predictions = model.predict(x = test_batch, verbose = 0)
    cm = confusion_matrix(y_true=test_batch.classes, y_pred = np.argmax(predictions, axis = -1))
    return plot_confusion_matrix(cm = cm, classes = cm_plot_labels, title = 'Confusion Matrix')

In [36]:
bird_classifier.evaluate(X_val, y_val)

15/15 [==============================] - 86s 6s/step - loss: 0.3216 - accuracy: 0.8812


[0.3216049373149872, 0.8812094926834106]

In [37]:
bird_classifier.evaluate(X_test, y_test)

7/7 [==============================] - 32s 4s/step - loss: 0.9965 - accuracy: 0.6080


[0.9965476989746094, 0.6080402135848999]